In [1]:
import pandas as pd

In [2]:
webpage1 = pd.read_html('https://www.fifaindex.com/teams/fifa20_419/?league=308&order=desc')
webpage1

[    Unnamed: 0                        Name    League   ATT   MID   DEF   OVR  \
 0          NaN                         NaN       NaN   NaN   NaN   NaN   NaN   
 1          NaN                         NaN       NaN   NaN   NaN   NaN   NaN   
 2          NaN                  SL Benfica  Liga NOS  78.0  80.0  80.0  79.0   
 3          NaN                    FC Porto  Liga NOS  78.0  78.0  82.0  79.0   
 4          NaN                    SC Braga  Liga NOS  77.0  77.0  76.0  76.0   
 5          NaN                 Sporting CP  Liga NOS  74.0  77.0  78.0  76.0   
 6          NaN           Vitória Guimarães  Liga NOS  72.0  74.0  75.0  74.0   
 7          NaN                         NaN       NaN   NaN   NaN   NaN   NaN   
 8          NaN                         NaN       NaN   NaN   NaN   NaN   NaN   
 9          NaN                  Rio Ave FC  Liga NOS  73.0  74.0  73.0  73.0   
 10         NaN  Futebol Clube de Famalicão  Liga NOS  74.0  74.0  71.0  72.0   
 11         NaN             

In [6]:
webpage1[0]

,Unnamed: 0,Name,League,ATT,MID,DEF,OVR,Team Rating,Unnamed: 8,Unnamed: 9,...,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98,Unnamed: 99
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,SL Benfica,Liga NOS,78.0,80.0,80.0,79.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,FC Porto,Liga NOS,78.0,78.0,82.0,79.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,SC Braga,Liga NOS,77.0,77.0,76.0,76.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,Sporting CP,Liga NOS,74.0,77.0,78.0,76.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,Vitória Guimarães,Liga NOS,72.0,74.0,75.0,74.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,Rio Ave FC,Liga NOS,73.0,74.0,73.0,73.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df = pd.DataFrame(webpage1[0])
df

,Unnamed: 0,Name,League,ATT,MID,DEF,OVR,Team Rating,Unnamed: 8,Unnamed: 9,...,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98,Unnamed: 99
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,SL Benfica,Liga NOS,78.0,80.0,80.0,79.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,FC Porto,Liga NOS,78.0,78.0,82.0,79.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,SC Braga,Liga NOS,77.0,77.0,76.0,76.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,Sporting CP,Liga NOS,74.0,77.0,78.0,76.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,Vitória Guimarães,Liga NOS,72.0,74.0,75.0,74.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,Rio Ave FC,Liga NOS,73.0,74.0,73.0,73.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df.dropna(subset = ["Name"], inplace=True)
df

,Unnamed: 0,Name,League,ATT,MID,DEF,OVR,Team Rating,Unnamed: 8,Unnamed: 9,...,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98,Unnamed: 99
2,NaN,SL Benfica,Liga NOS,78.0,80.0,80.0,79.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,FC Porto,Liga NOS,78.0,78.0,82.0,79.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,SC Braga,Liga NOS,77.0,77.0,76.0,76.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,Sporting CP,Liga NOS,74.0,77.0,78.0,76.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,Vitória Guimarães,Liga NOS,72.0,74.0,75.0,74.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,Rio Ave FC,Liga NOS,73.0,74.0,73.0,73.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,Futebol Clube de Famalicão,Liga NOS,74.0,74.0,71.0,72.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,NaN,Moreirense FC,Liga NOS,72.0,71.0,71.0,71.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,NaN,Gil Vicente FC,Liga NOS,71.0,71.0,70.0,71.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,NaN,Boavista FC,Liga NOS,70.0,71.0,71.0,71.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df.drop(df.columns.difference(['Name','League','ATT','MID','DEF','OVR']), 1, inplace=True)
df

,Name,League,ATT,MID,DEF,OVR
2,SL Benfica,Liga NOS,78.0,80.0,80.0,79.0
3,FC Porto,Liga NOS,78.0,78.0,82.0,79.0
4,SC Braga,Liga NOS,77.0,77.0,76.0,76.0
5,Sporting CP,Liga NOS,74.0,77.0,78.0,76.0
6,Vitória Guimarães,Liga NOS,72.0,74.0,75.0,74.0
9,Rio Ave FC,Liga NOS,73.0,74.0,73.0,73.0
10,Futebol Clube de Famalicão,Liga NOS,74.0,74.0,71.0,72.0
11,Moreirense FC,Liga NOS,72.0,71.0,71.0,71.0
12,Gil Vicente FC,Liga NOS,71.0,71.0,70.0,71.0
13,Boavista FC,Liga NOS,70.0,71.0,71.0,71.0
